# IFC Export using Autodesk Automation API - Revit 2025

This notebook demonstrates how to export IFC files from Revit 2025 models using the Autodesk Platform Services (APS) Design Automation API.

**Important:** This example embeds JSON configuration data directly in the work item payload using `ActivityJsonParameter`. This approach works well for smaller JSON files (under 60KB).

**Note:** The input Revit file, JSON export settings, and compiled app bundle can be found in `files/` subdirectory.

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [ ]:
import os
import json
import logging
import uuid
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    ActivityJsonParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step2-create-forge-app/

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [2]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned.
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

Authentication successful. Nickname: viktortest


## Step 3: Define Revit 2025 Configuration

Set up constants for the Revit 2025 IFC export workflow.

**Note:** For other Revit versions, change the year suffix and engine version accordingly.

In [3]:
# Define constants for Revit 2025
# For other versions, update the year suffix (2023, 2024, 2025, 2026)
year = 2025
app_bundle_base_name = "IfcExportBundle"
activity_base_name = "RevitIfcExportAppActivity"
alias = "prod"
bucket_key = uuid.uuid4().hex

# Add year suffix to names
app_bundle_name = f"{app_bundle_base_name}{year}"
activity_name = f"{activity_base_name}{year}"

# Path to the bundle ZIP file (relative to this notebook's directory)
zip_path = Path.cwd() / "files" / "IfcExportDA.bundle.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"Revit Version: {year}")
print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")
print(f"ZIP Path: {zip_path}")

Revit Version: 2025
App Bundle: viktortest.IfcExportBundle2025+prod
Activity: viktortest.RevitIfcExportAppActivity2025+prod
ZIP Path: c:\Users\aleja\viktor-apps\add-type-parameter-aps\autodesk_automation - ExportIFC\2025 - 2026 - Net8\files\IfcExportDA.bundle.zip


## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the IFC export logic.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step4-publish-appbundle/

**Note:** For other Revit versions, change the engine version accordingly (e.g., Autodesk.Revit+2024, Autodesk.Revit+2026).

In [4]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine=f"Autodesk.Revit+{year}",  # For Revit 2024: Autodesk.Revit+2024, For 2026: Autodesk.Revit+2026, etc.
    alias=alias,
    zip_path=str(zip_path),
    description=f"Exports IFC from selected views for Revit {year}",
)

bundle.deploy(token)
print(f"✓ App bundle deployed successfully for Revit {year}!")

✓ App bundle deployed successfully for Revit 2025!


## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

**Note:** This example uses `ActivityJsonParameter` to embed JSON settings directly in the work item payload. This is suitable for configuration data under 60KB.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/#step-1-create-a-new-activity

In [5]:
# Input: Revit file
input_revit = ActivityInputParameter(
    name="rvtFile",
    localName="input.rvt",
    verb="get",
    description="Input Revit File",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="input.rvt",
)

# Output: Zipped IFC files
output_zip = ActivityOutputParameter(
    name="result",
    localName="result",
    verb="put",
    description="Zipped IFCs",
    zip=True,
    bucketKey=bucket_key,
    objectKey="IFCExport.zip",
)

# Input: JSON settings
input_json = ActivityJsonParameter(
    name="ifcSettings",
    localName="ifc_settings.json",
    verb="get",
    description="Export Settings Parameter JSON",
)

print("Activity parameters defined successfully!")

Activity parameters defined successfully!


## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/

In [6]:
activity = Activity(
    id=activity_name,
    parameters=[input_revit, output_zip, input_json],
    engine=f"Autodesk.Revit+{year}",  # For Revit 2024: Autodesk.Revit+2024, For 2026: Autodesk.Revit+2026, etc.
    appbundle_full_name=appbundle_full_alias,
    description=f"Export IFC from views using JSON settings for Revit {year}",
    alias=alias,
)

# Set the Revit command
activity.set_revit_command_line()
activity.deploy(token=token)
print(f"✓ Activity deployed successfully for Revit {year}!")

✓ Activity deployed successfully for Revit 2025!


## Step 7: Upload Input Revit File

Upload the Revit file to Object Storage Service (OSS).

**Note:** Input files should be in the `files/` subdirectory relative to this notebook.

In [7]:
# Path to input Revit file (in files/ subdirectory)
input_rvt_path = Path.cwd() / "files" / "sample_model_2025.rvt"

if not input_rvt_path.exists():
    raise FileNotFoundError(f"Input Revit file not found: {input_rvt_path}")

input_revit.upload_file_to_oss(file_path=str(input_rvt_path), token=token)
print(f"✅ Input Revit file uploaded: {input_rvt_path.name}")

✅ Input Revit file uploaded: sample_model_2025.rvt


In [8]:
# Load IFC export settings from JSON file (in files/ subdirectory)
settings_path = Path.cwd() / "files" / "export_settings.json"

if not settings_path.exists():
    raise FileNotFoundError(f"Export settings file not found: {settings_path}")

with settings_path.open("r", encoding="utf-8") as f:
    settings = json.load(f)

input_json.set_content(settings)
print(f"✅ IFC settings loaded from: {settings_path.name}")
print(f"Settings: {json.dumps(settings, indent=2)}")

✅ IFC settings loaded from: export_settings.json
Settings: {
  "view_names": [
    "{3D}"
  ],
  "FileVersion": "IFC4",
  "IFCFileType": "IFC",
  "ExportBaseQuantities": true,
  "SpaceBoundaryLevel": 2,
  "FamilyMappingFile": "C:\\Data\\ifc_mapping.txt",
  "ExportInternalRevitPropertySets": false,
  "ExportIFCCommonPropertySets": true,
  "ExportAnnotations": false,
  "Export2DElements": false,
  "ExportRoomsInView": false,
  "VisibleElementsOfCurrentView": false,
  "ExportLinkedFiles": false,
  "IncludeSteelElements": false,
  "ExportPartsAsBuildingElements": true,
  "UseActiveViewGeometry": false,
  "UseFamilyAndTypeNameForReference": false,
  "Use2DRoomBoundaryForVolume": false,
  "IncludeSiteElevation": false,
  "ExportBoundingBox": false,
  "ExportSolidModelRep": false,
  "StoreIFCGUID": false,
  "ExportSchedulesAsPsets": false,
  "ExportSpecificSchedules": false,
  "ExportUserDefinedPsets": false,
  "ExportUserDefinedPsetsFileName": "",
  "ExportUserDefinedParameterMapping": false

## Step 9: Create and Execute Work Item

Create a work item with all parameters and execute it.

In [9]:
work_item = WorkItem(
    parameters=[input_revit, output_zip, input_json],
    activity_full_alias=activity_full_alias
)

print("Work item created. Starting execution...")
print("This may take several minutes...")

Work item created. Starting execution...
This may take several minutes...


## Step 10: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [10]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print("=" * 60)
print(f"Work item completed with status: {last_status}")
print("Full response:")
print(json.dumps(status_resp, indent=2))

2025-11-25 13:58:04,391 INFO Polling work item status, id=40c1f837dffd43caa9646761cdfede1f
2025-11-25 13:58:05,244 INFO [  0s] status=pending report_url=None
2025-11-25 13:58:05,244 INFO [  0s] status=pending report_url=None
2025-11-25 13:58:16,046 INFO [ 10s] status=pending report_url=None
2025-11-25 13:58:16,046 INFO [ 10s] status=pending report_url=None
2025-11-25 13:58:26,856 INFO [ 20s] status=pending report_url=None
2025-11-25 13:58:26,856 INFO [ 20s] status=pending report_url=None
2025-11-25 13:58:37,664 INFO [ 30s] status=pending report_url=None
2025-11-25 13:58:37,664 INFO [ 30s] status=pending report_url=None
2025-11-25 13:58:48,476 INFO [ 40s] status=pending report_url=None
2025-11-25 13:58:48,476 INFO [ 40s] status=pending report_url=None
2025-11-25 13:58:59,347 INFO [ 50s] status=inprogress report_url=None
2025-11-25 13:58:59,347 INFO [ 50s] status=inprogress report_url=None
2025-11-25 13:59:10,180 INFO [ 60s] status=inprogress report_url=None
2025-11-25 13:59:10,180 INFO 

KeyboardInterrupt: 

## Step 11: Download Results

If successful, download the IFC export results to the `files/output/` subdirectory.

In [ ]:
if last_status == "success":
    print("\n" + "=" * 60)
    print("Downloading results...")
    out_dir = Path.cwd() / "files" / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_zip = out_dir / "IFCExport.zip"
    
    output_zip.download_to(output_path=str(out_zip), token=token)
    print(f"✅ Download successful: {out_zip}")
    print("\n🎉 IFC Export workflow completed successfully!")
else:
    print(f"\n❌ Work item failed with status: {last_status}")
    print("Check the response above for error details.")

## Step 12: Cleanup Resources (Optional)

Delete the activity and app bundle to clean up resources.

In [ ]:
print("Cleaning up resources...")

# Delete activity
try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✓ Deleted activity for Revit {year}: {activity_full_alias}")
except Exception as e:
    print(f"✗ Failed to delete activity for Revit {year}: {e}")

# Delete app bundle
try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✓ Deleted app bundle for Revit {year}: {appbundle_full_alias}")
except Exception as e:
    print(f"✗ Failed to delete app bundle for Revit {year}: {e}")

print("\nCleanup completed!")